<a href="https://colab.research.google.com/github/SahanaRaghu/myWork_log_analysis/blob/master/Analyse_myWorkLogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download log file from Google drive & load to csv

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [75]:
listed = drive.ListFile({'q': "title contains 'myWork_logs_url_null.json' and '1Ea4CQzbR1PTeM0aMDEknCVa2xpagiBRf' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title myWork_logs_url_null.json, id 1U_gToZS6AnAv-qa40ordZp2OkuvBzGqx


In [0]:
import os
import pandas as pd

download_path = os.path.expanduser('~/data_logs')
try:
  os.makedirs(download_path)
except FileExistsError:
  pass

In [0]:
output_file = os.path.join(download_path, 'logs.json')
temp_file = drive.CreateFile({'id': '1U_gToZS6AnAv-qa40ordZp2OkuvBzGqx'})
temp_file.GetContentFile(output_file)

In [78]:
output_file

'/root/data_logs/logs.json'

In [0]:
import json
from pprint import pprint

with open(output_file) as f:
    json_data = json.load(f)

In [0]:
good_cols = [
 '@timestamp',
 '@version',
 'acorn_app',
 'acorn_appVersion',
 'acorn_deviceTime_orApp',
 'acorn_deviceid',
 'acorn_logapp',
 'acorn_message',
 'acorn_pid',
 'acorn_store',
 'acorn_thread',
 'acorn_timestamp',
 'acorn_type',
 'beat',
 'code',
 'host',
 'input_type',
 'message',
 'offset',
 'source',
 'tags',
 'type'
]

In [0]:
data=[]
for each_data in json_data['hits']:
  selected_row = []
  for item in good_cols:
    selected_row.append(each_data['_source'][item])
  data.append(selected_row)


In [0]:
import pandas as pd

data = pd.DataFrame(data, columns=good_cols)

### Feature Engeneering

In [0]:
data_backup = data.copy()

In [0]:
data = data.drop(['@version', 'acorn_logapp', 'acorn_type', 'input_type', 'source', 'type', 'acorn_pid', 'acorn_thread', 'code', 'host', 'acorn_timestamp'],axis=1)

In [86]:
data.columns

Index(['@timestamp', 'acorn_app', 'acorn_appVersion', 'acorn_deviceTime_orApp',
       'acorn_deviceid', 'acorn_message', 'acorn_store', 'acorn_timestamp',
       'beat', 'message', 'offset', 'tags'],
      dtype='object')

In [0]:
data.sort_values(by=['acorn_deviceTime_orApp'])


In [88]:
data.head()

,@timestamp,acorn_app,acorn_appVersion,acorn_deviceTime_orApp,acorn_deviceid,acorn_message,acorn_store,acorn_timestamp,beat,message,offset,tags
0,2019-03-01T08:50:13.657Z,myWork-backstock-take,1.0.20,1551430208238,94fb294e733c,Sending message AUTHENTICATE to gatekeeper..|,t2660,03-01-2019 02:50:13.512,"{'hostname': 'TCTTSMYD4P', 'name': 'TCTTSMYD4P...",03-01-2019 02:50:13.512 -06|Info|Central|12488...,31205165,"[storesCentralPerf, beats_input_codec_plain_ap..."
1,2019-03-01T08:50:13.657Z,myWork-backstock-take,1.0.20,1551430208238,94fb294e733c,Service is not bound|,t2660,03-01-2019 02:50:13.512,"{'name': 'TCTTSMYD4P', 'version': '5.4.1', 'ho...",03-01-2019 02:50:13.512 -06|Info|Central|12488...,31205308,"[storesCentralPerf, beats_input_codec_plain_ap..."
2,2019-03-01T08:50:13.656Z,myWork-backstock-take,1.0.20,1551430208236,94fb294e733c,Sending message AUTHENTICATE to gatekeeper..|,t2660,03-01-2019 02:50:13.512,"{'hostname': 'TCTTSMYD4P', 'name': 'TCTTSMYD4P...",03-01-2019 02:50:13.512 -06|Info|Central|12488...,31203700,"[storesCentralPerf, beats_input_codec_plain_ap..."
3,2019-03-01T08:50:13.656Z,myWork-backstock-take,1.0.20,1551430208236,94fb294e733c,Service is not bound|,t2660,03-01-2019 02:50:13.512,"{'version': '5.4.1', 'name': 'TCTTSMYD4P', 'ho...",03-01-2019 02:50:13.512 -06|Info|Central|12488...,31203843,"[storesCentralPerf, beats_input_codec_plain_ap..."
4,2019-03-01T08:50:13.656Z,myWork-backstock-take,1.0.20,1551430208237,94fb294e733c,java.lang.IllegalStateException: Service is no...,t2660,03-01-2019 02:50:13.512,"{'name': 'TCTTSMYD4P', 'hostname': 'TCTTSMYD4P...",03-01-2019 02:50:13.512 -06|Info|Central|12488...,31204998,"[storesCentralPerf, beats_input_codec_plain_ap..."
